%% import necessary packages. explain category encoder

In [ ]:
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVR
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OrdinalEncoder
import category_encoders as ce

In [ ]:
dataset = results_df

%% data cleaning

In [ ]:
test = dataset.sample(n=1000)

In [ ]:
check = dataset['response duration'].isnull()
print(check.value_counts())

In [ ]:
check = dataset['travel time'].isnull()
temp = dataset['travel time'].value_counts()
print(check.value_counts())

In [ ]:
temp = dataset['call_Type'].value_counts()

In [ ]:
goodMean = dataset['response duration'].mean(skipna = True)
dataset['response duration'] = dataset['response duration'].fillna(goodMean)

In [ ]:
goodMean = dataset['travel time'].mean(skipna = True)
dataset['travel time'] = dataset['travel time'].fillna(goodMean)

In [ ]:
dataset_clean = dataset[dataset['response duration']<15]

In [ ]:
dataset.columns
 
# %% visualizations

In [ ]:
temp = dataset[dataset['response duration'].between(0,15)]
plt.hist(temp['response duration'],bins = 30)

In [ ]:
temp = dataset[dataset['travel time'].between(0,15)]
plt.hist(temp['travel time'],bins = 30)

%% encoding<br>
=============================================================================<br>
one-hot encode entire datasets BEFORE train/test split so we don't end up with train/test sets with different numbe of columns<br>
=============================================================================

In [ ]:
X = dataset.drop(['travel time','response duration','dispatch_dttm','response_dttm'], axis=1)
 
encoder = OneHotEncoder(handle_unknown = 'ignore')
X_enc = encoder.fit_transform(X)
#Xopt_enc = encoder.fit_transform(X_opt)
 
encoder.categories_
 
y = dataset["response duration"].values
df_map = pd.DataFrame({'original column':X.columns,
                       'categories':encoder.categories_})

%% train/test split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_enc, y, test_size=0.2, random_state=42)

_train, X_test, y_train, y_test = train_test_split(Xopt_enc, y, test_size=0.2, random_state=42)

In [ ]:
y_train_inv = -1*y_train

In [ ]:
print(X_train.shape)
print(X_test.shape)

%% obtain baseline predictive model, by just always predicting the average

In [ ]:
avgDelay = np.full(y_test.shape, np.mean(y_train), dtype=float)

In [ ]:
avgDelay

In [ ]:
mean_squared_error(y_test, avgDelay)

%% create models and queue

In [ ]:
if 'scores' not in locals():
    scores = pd.DataFrame({'model':[],
              'score':[]})

In [ ]:
verbosity = 3

In [ ]:
lr = linear_model.LinearRegression(fit_intercept=True, normalize=True)
sgdr = linear_model.SGDRegressor(max_iter=1000, tol =1e-3, verbose = 3)
ridge = linear_model.Ridge(alpha = 0.01)
ridge = linear_model.Ridge(alpha = 1)
ridge_cv = linear_model.RidgeCV(alphas = [1e-1,1e0,1e1])
lasso = linear_model.Lasso()
lasso_cv = linear_model.LassoCV(alphas = [1e-3,1e-2,1e-1,1e0,1e1,1e2,1e3], verbose = 3)
linSVR1 = LinearSVR(C=0.1, verbose = 3)
linSVR2 = LinearSVR(C=1, verbose = 3)
linSVR3 = LinearSVR(C=10, verbose = 3)

In [ ]:
models = [lr,  sgdr,ridge,ridge_cv,lasso, lasso_cv]
models = [lr,  sgdr,ridge, lasso, linSVR1]
models = [ridge]

odels = [lr, sgdr,ridge,lasso]<br>
odels = [linSVR1, linSVR2, linSVR3]<br>
odels = [lr]

In [ ]:
					   
importances = pd.DataFrame({'feature':encoder.get_feature_names()})
importances['feature category'] = importances['feature'].str[1]

%% fit models to training set

In [ ]:
for i in range(len(models)):
    print('fitting ',models[i])
    
    models[i].fit(X_train,y_train)
    #models[i].fit(X_train,y_train_inv)
    
    models[i].predict(X_test)
    modelScore = models[i].score(X_test, y_test)
    scores = scores.append({'model':str(models[i].__class__.__name__),
                              'score':modelScore},
                              ignore_index = True)
# %% tune hyperparameters of linear SVR

In [ ]:
turnon = False
if turnon == True:
    linSVRpreop = LinearSVR(verbose = 3, max_iter = 2000)
    parameters = {'epsilon':[0.1, 1, 10], 'C':[0.1, 1, 10]}
    svrOpt = GridSearchCV(linSVRpreop,parameters)
    
    svrOpt.fit(X_train,y_train)
                      
# %% compile feature importances

In [ ]:
for i in range(len(models)):
    print('calculating feature importances for: ', models[i])
    feIm = permutation_importance(models[i],X_train.toarray(), y_train, n_repeats = 5)
    importances[str(models[i])] = feIm.importances_mean

In [ ]:
importances_pivot = pd.pivot_table(importances, index = 'feature category',aggfunc = 'mean')

%% some more visualizations

In [ ]:
def plotAgainst(dataset,column, y_axis):
    dataset_piv = pd.pivot_table(dataset, index = column)
    dataset_piv = dataset_piv.reset_index()
    dataset_piv = dataset_piv.sort_values(by = y_axis)
    plt.scatter(dataset_piv[column],dataset_piv[y_axis])
    label = (y_axis, ' vs. ', column)
    plt.title(label)

In [ ]:
for i in range(len(df_map)):
    ds_fields = df_map['original column'].tolist()
    plt.figure()
    plotAgainst(dataset,ds_fields[i],'response duration') 
    plt.show()

%% try an ordinal encoder

In [ ]:
ordinalMapping = [{'col':'original_priority',
                   'mapping':
                            {'2':1,
                            'B':2,
                            'C':3,
                            'A':4,
                            'E':5,
                            '3':6,
                            'I':7}},
                     {'col':'priority',
                      'mapping':{
                              '2':1,
                              'E':2,
                              '3':3}},
                      {'col':'number_of_alarms',
                       'mapping':
                           {'1':1,
                            '2':2}
                              }]
                    
                                  

In [ ]:
ordEnc = ce.OrdinalEncoder(mapping = ordinalMapping)
X_opt = ordEnc.fit_transform(X)
X_opt = X_opt.astype({'original_priority':'int64','priority':'int64', 'number_of_alarms':'int64'})